## CONFIGURE PRIMARY CLUSTER

In [1]:
%env  WORKDIR=/tmp/vault

env: WORKDIR=/tmp/vault


In [2]:
%%bash
gcloud auth login
export VAULT_ADDR=$(terraform output -raw cluster_primary_fqdn_8200)
vault_token_primary=$(terraform output -raw cluster_primary_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_primary")

vault write -f sys/replication/performance/primary/enable primary_cluster_addr=$(terraform output -raw cluster_primary_fqdn_8201)
sleep 5
vault write -format=json sys/replication/performance/primary/secondary-token id=pr12 | jq -r .wrap_info.token > $WORKDIR/pr_token.txt

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=2RVC2EWouOdVlytPal8pyI38mdWWJs&access_type=offline&code_challenge=KJfSTU2Yrsk2scA27IFDaSP6T3TxKRXFwwSJwT8CdvU&code_challenge_method=S256


You are now logged in as [jose.merchan@hashicorp.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
WARNING! The following warnings were returned from Vault:

  * This cluster is being enabled as a primary for replication. Vault will 

## CONFIGURE SECONDARY CLUSTER

In [3]:
%%bash

export VAULT_ADDR=$(terraform output -raw cluster_pr_fqdn_8200)
vault_token_pr=$(terraform output -raw cluster_pr_read_vault_token)
# Execute the command stored in $vault_token
export VAULT_TOKEN=$(eval "$vault_token_pr")

vault write sys/replication/performance/secondary/enable  \
    primary_api_addr=$(terraform output -raw cluster_primary_fqdn_8200) \
    token=$(cat $WORKDIR/pr_token.txt)

WARNING! The following warnings were returned from Vault:

  * Vault has successfully found secondary information; it may take a while to
  perform setup tasks. Vault will be unavailable until these tasks and initial
  sync complete.

